<a href="https://colab.research.google.com/github/dlwub/CLTV_Computation/blob/main/CLTV_Computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pickle
import os

# Takes input from the user and returns the Marcov Matrix
def Marcov_matrix():
    global n
    global P
    global p
    n=int(input('Insert the absorbing state:\n'))
    while n<=1 or not isinstance(n,int):
        print('Invalid Input')
        n=input('Insert the absorbing state again:\n')
    P=np.zeros((n,n)) #Creates nxn zero matrix
    p=[]
    i=0
    while i <n-1: # Receives probabilities from the user and forms the Marcov Matrix
        x=float(input('Insert probability p%d (probability customer buys in the current period given the customer is in recency state %d last period:\n'%(i+1,i+1)))
        if 0<= x<= 1:# Checks if the given probabilities are between 0 and 1 and in decreasing order.
            p.append(x)
            P[i][0]=x
            P[i][i+1]=1-x
            if i>0:
                if p[i]< p[i-1]:
                    i=i+1
                else:
                    del p[i]
                    print('Invalid input')
                    i=i #if the input is not valid, requests for an input again
            else:
                i=i+1
        else:
            print(' Invalid input')
            i=i  #if the input is not valid, requests for an input again
    p.append(0)
    P[n-1][n-1]=1
    return P

# Takes input from the user and returns the Payoff Matrix
def Payoff_matrix():
    global r
    global MC
    global M
    global G
    r=float(input('Insert the discount rate that you apply:\n'))
    while 0>r or r>1: #Checks if 0<r<1
        print('Invalid input')
        r=float(input('Insert the discount rate that you apply again:\n'))
    MC=float(input('Insert the marginal contribution that a customer generates if he/she purchase:\n'))
    M=float(input('Insert the amount the firm spends to make a customer repurchase:\n'))
    while M<0:
        print('Invalid input')
        M=float(input('Insert the amount the firm spends to make a customer repurchase again:\n'))
    G=np.zeros((n,1)) #Forms nx1 zero matrix
    G[0][0]=MC-M
    j=1 # Forms the payoff matrix
    while j<n-1:
        G[j][0]=-M if M>0 else 0
        j=j+1
    G[n-1][0]=0
    return G

# Matrix mutiplication of two matrices in this program(not arbitrary matrices)
def matrix_product(A,B):
    C=np.zeros((len(A[:,0]),len(B[0,:])))
    l=0
    while l<len(A[:,0]):
        m=0
        while m <len(B[0,:]):
            C[l][m]=np.dot(A[l,:],B[:,m])
            m=m+1
        l=l+1
    return C

def matrix_power(P,t):#Multiplies a square matrix of dimension n with itself t-times
    if t==0:
        C=np.identity(n)
    else:
        C=matrix_product(matrix_power(P,t-1),P)
    return C

# Prints the table heading
def print_heading():
    print('-'*167)
    print(' '*60,'Migration Model Calculation')
    print(' '*10,'Recency state(j)')
    print(' '*7,'-'*(7*n))
    print('j=','\t')
    j=1
    while j< n:
        print(j,'\t')
        j=j+1
    print('>=',n,'\t'*(n+4),'Delta=',r)
    print('pj=','\t')
    j=0
    while j<n:
        print(p[j],'\t')
        j=j+1
    print('\t'*(n+2),'Marginal','\t','Expected','\t','Discounted')
    print('-'*7*(n+1),'\t'*(n+3),'contribution','\t','profit','\t'*2,'expected')
    print('Period','\t')
    j=1
    while j<n:
        print(j,'\t'*2)
        j=j+1
    print('>=',n,'\t'*2,'Purch t','\t','if purchase',3*'\t','profit')
    print('-'*167)

# Prints the body of the table

def build_body():
    global F
    F=np.zeros((20,n+4))
    i=0
    while i< 20:
        D=matrix_power(P,i)

        j=0
        while j< n:
            F[i][j]=D[0,j]

            j=j+1
        F[i][n]= D[0,0]
        F[i][n+1]= MC
        F[i][n+2]= F[i][n]*MC
        F[i][n+3]=((1/(1+r)**i)*F[i][n]*MC)

        i=i+1
def print_body():
    i=0
    while i< 20:
        print(i+1,'\t')
        j=0
        while j< n:
            print("%.6f"%F[i][j],'\t')
            j=j+1
        print("%.6f"%F[i][n],'\t','$',"%.6f"%F[i][n+1],'\t','$',"%.6f"%F[i][n+2],'\t','$',"%.6f"%F[i][n+3])
        i=i+1
# Prints the table showing Migration Model Calculation
def print_table():
    print_heading()
    print_body()

# Calculates the total net present value for 20 periods and prints the results
def total_npv():
    global s
    q=0
    s=0
    while q< 20:
        V=matrix_product (((1+r)**-q)*matrix_power(P,q),G)
        s=s+V
        q=q+1
    return s

# Calculates the total life time value of a customer and returns the result
def CLTV():
    global Vinf
    Vinf=matrix_product(np.linalg.inv(np.identity(n)-(1/(1+r))*P),G)
    return Vinf

# Saves the table heading into a permanent file
def save_heading():
    pickle.dump('-'*167,fout)
    pickle.dump(' '*60 + 'Migration Model Calculation',fout)
    pickle.dump(' '*10 + 'Recency state(j)',fout)
    pickle.dump(' '*7 + '-'*(7*n),fout)
    fout.write(b'j=\t')
    j=1
    while j< n:
        fout.write(b'%d\t'%j)
        j=j+1
    fout.write(b'>=')
    fout.write(b'%d'%n)
    fout.write(b'\t'*(n+4))
    fout.write(b'Delta=%g\n'%r)
    fout.write(b'pj=\t')
    j=0
    while j<n:
        fout.write(b'%g\t'%p[j])
        j=j+1
    fout.write(b'\t'*(n+2))
    fout.write(b'Marginal\t')
    fout.write(b'Expected\t')
    fout.write(b'Discounted\n')
    fout.write(b'-'*7*(n+1))
    fout.write(b'\t'*(n+3))
    fout.write(b'contribution\t')
    fout.write(b'profit\t\t')
    fout.write(b'expected\n')
    fout.write(b'Period\t')
    j=1
    while j<n:
        fout.write(b'%d\t\t'%j)
        j=j+1
    fout.write(b'>=')
    fout.write(b'%d\t\t'%n)
    fout.write(b'Purch t\t\t')
    fout.write(b'if purchase\t\t\t')
    fout.write(b'profit\n')
    fout.write(b'-'*167)

# Saves the body of the table into a permanent file
def save_body():
    i=0
    while i< 20:
        fout.write(b'%d\t'%(i+1))
        j=0
        while j< n:
            fout.write(b"%.6f"%float('%g'%F[i][j])+b'\t')
            j=j+1
        fout.write(b"%.6f"%float('%g'%F[i][n])+b'\t$')
        fout.write(b"%.6f"%float('%g'%F[i][n+1])+b'\t$')
        fout.write(b"%.6f"%float('%g'%F[i][n+2])+b'\t$')
        fout.write(b"%.6f"%float('%g'%F[i][n+3])+b'\n')
        i=i+1

# Saves the table into a permanent file
def save_table():
    save_heading()
    save_body()

# Saves data for permanent storage
def save_file():
    global fout
    output=open('prev_data.txt','wb')# The purpose of this file is only to enable the program
                                     # to check if it is being used for the first time or not
    output.close()
    fout=open('%s.txt' % file_name,'wb')# Write to file in binary mode
    pickle.dump(n,fout)
    fout.write(b'The Marcov matrix is:\n')
    pickle.dump(P,fout)
    fout.write(b'\n')
    fout.write(b'The payoff matrix is:\n')
    pickle.dump(G,fout)
    save_table()
    fout.write(b'-'*334+b'\n')
    fout.write(b'The total NPV for 20 periods obtainable from the customer is:\n')
    pickle.dump(s,fout)
    fout.write(b'The CLTV is:\n')
    pickle.dump(Vinf,fout)
    fout.write(b'\n')
    print('File saved')
    fout.close()

# Receives file name for storage
def pre_saving():
    global file_name
    file_name=input('Insert a file name for permanent storage of the data:\n')
    if os.path.exists('%s.txt'%file_name)==True:
        choice=input('There is a file with name %s.txt.Do you want to overwrite it?Y/N:\n'%file_name)
        if choice=='Y':
            save_file()
        elif choice=='N':
            del file_name
            pre_saving()
        else:
            print('Invalid choice')
            del file_name
            pre_saving()
    else:
        save_file()

# Reads a file and displays its content
def read_file():
    n=pickle.load(fin)
# Reads the Marcov matrix and the payoff matrix
    print(fin.readline())
    print(pickle.load(fin))
    print(fin.readline())
    print(fin.readline())
    print(pickle.load(fin))
# Reads the heading of the table
    for i in range(1,5):
        print(pickle.load(fin))
    for i in range(1,6):
        print(fin.readline())
# Reads the body of the table
    for i in range(1,20):
        print(fin.readline())
    print(fin.readline())
# Reads the total NPV for 20 periods and the CLTV
    for i in range(1,3):
        print(fin.readline())
        print(pickle.load(fin))
    print(fin.readline())

# Receives file name for reading
def pre_reading():
    global fin
    file_name2=input('Insert the name of the file you want to read:\n')
    if os.path.exists('%s.txt'%file_name2)==False:
        print('There is no file named %s.txt.'%file_name2)
        read_file()
    else:
        fin=open('%s.txt'%file_name2,'rb')# Read file in binary mode
        read_file()
        fin.close()
        exit_program()

def MCM_computation():
    Marcov_matrix()
    Payoff_matrix()
    total_npv()
    CLTV()
    build_body()
    user_interface()
    pre_saving()
    exit_program()

# Opens the program
def open_program():
    if os.path.exists('prev_data.txt')==False:
        print('Welcome to MCM computation.')
        MCM_computation()
    else:
        option=input('What do you want to do? Do MCM computation or Read existing data?:C/R\n')
        if option=='C':
            MCM_computation()
        elif option=='R':
            pre_reading()
        else:
            print('Invalid choice.')
            open_program()

def more_print():
    choice=input('Do you want to print something else:Y/N\n')
    if choice=='Y':
        user_interface()
    elif choice=='N':
        return
    else:
        print('Invalid input')
        more_print()

def user_interface():
    print('What do you want to print?\n')
    key_press=input('Press M for Marcov matrix, G for pay off matrix, T for table, N for NPV and C for CLTV:\n')
    if key_press=='M':
        print('The Marcov matrix is:\n\n',P,'\n')
    elif key_press=='G':
        print('The payoff matrix is:\n\n',G)
    elif key_press=='T':
        print_table()
        print('-'*334,'\n')
    elif key_press=='N':
        total_npv()
        print('The total NPV for 20 periods obtainable from the customer is:\n\n',s,'\n')
    elif key_press=='C':
        CLTV()
        print('The CLTV is:\n\n',Vinf,'\n')
    else:
        print('Invalid input')
        user_interface()
    more_print()

def exit_program():
    option=input('Do you want another operation?Y/N:\n')
    if option=='Y':
        open_program()
    elif option=='N':
        print('Thank you!')
    else:
        exit_program()

open_program()



What do you want to do? Do MCM computation or Read existing data?:C/R
C
Insert the absorbing state:
3
Insert probability p1 (probability customer buys in the current period given the customer is in recency state 1 last period:
.6
Insert probability p2 (probability customer buys in the current period given the customer is in recency state 2 last period:
.35
Insert the discount rate that you apply:
.7
Insert the marginal contribution that a customer generates if he/she purchase:
12
Insert the amount the firm spends to make a customer repurchase:
4
What do you want to print?

Press M for Marcov matrix, G for pay off matrix, T for table, N for NPV and C for CLTV:
M
The Marcov matrix is:

 [[0.6  0.4  0.  ]
 [0.35 0.   0.65]
 [0.   0.   1.  ]] 

Do you want to print something else:Y/N
Y
What do you want to print?

Press M for Marcov matrix, G for pay off matrix, T for table, N for NPV and C for CLTV:
G
The payoff matrix is:

 [[ 8.]
 [-4.]
 [ 0.]]
Do you want to print something else:Y/N
Y
W